In [2]:
!pip install joblib utils

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import joblib
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, classification_report, confusion_matrix,accuracy_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier


In [3]:
!pip install GEOparse
import GEOparse
gse = GEOparse.get_GEO("GSE6532", destdir="./")
df = gse.pivot_samples('VALUE')
print(df.head())

07-Jun-2025 11:32:10 DEBUG utils - Directory ./ already exists. Skipping.
07-Jun-2025 11:32:10 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE6nnn/GSE6532/soft/GSE6532_family.soft.gz to ./GSE6532_family.soft.gz


100%|██████████| 173M/173M [00:14<00:00, 12.5MB/s]   
07-Jun-2025 11:32:25 DEBUG downloader - Size validation passed
07-Jun-2025 11:32:25 DEBUG downloader - Moving C:\Users\vetdd\AppData\Local\Temp\tmpdvj2hx9b to C:\Users\vetdd\Documents\NSS_project\BreastCancerAIProject\notebook\GSE6532_family.soft.gz
07-Jun-2025 11:32:25 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE6nnn/GSE6532/soft/GSE6532_family.soft.gz
07-Jun-2025 11:32:25 INFO GEOparse - Parsing ./GSE6532_family.soft.gz: 
07-Jun-2025 11:32:25 DEBUG GEOparse - DATABASE: GeoMiame
07-Jun-2025 11:32:25 DEBUG GEOparse - SERIES: GSE6532
07-Jun-2025 11:32:25 DEBUG GEOparse - PLATFORM: GPL96
07-Jun-2025 11:32:26 DEBUG GEOparse - PLATFORM: GPL97
07-Jun-2025 11:32:27 DEBUG GEOparse - PLATFORM: GPL570
C:\Users\vetdd\anaconda3\Lib\site-packages\GEOparse\GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), i

name       GSM150794  GSM150795  GSM150796  GSM150797  GSM150798  GSM150799  \
ID_REF                                                                        
1007_s_at  11.605039  11.399484  11.344057  10.737436  11.363895  11.556413   
1053_at     7.194043   7.323444   7.368871   7.352203   7.243843   7.222124   
117_at      7.417318   7.133420   7.380305   7.374331   7.356792   7.423151   
121_at     10.136940  10.135816  10.008997   9.976584   9.787192   9.747580   
1255_g_at   5.259165   5.156951   4.989532   5.074146   4.920309   4.993376   

name       GSM150800  GSM150801  GSM150802  GSM150803  ...   GSM65871  \
ID_REF                                                 ...              
1007_s_at  11.375988  10.838380  11.146116  11.122378  ...  11.321648   
1053_at     7.209078   7.358861   6.923751   6.562159  ...   6.951810   
117_at      7.536636   7.497660   7.476072   7.507992  ...   7.097622   
121_at      9.859196   9.948644   9.253056   9.350627  ...   9.832048   
1255_g_a

In [4]:
st.set_page_config(page_title="Breast Cancer Risk Classifier", layout="centered")

st.title("🎀 Breast Cancer Risk Assessment App")
st.markdown("Upload patient data and assess risk level (low, medium, high)")

# Upload data
uploaded_file = st.file_uploader("Upload CSV patient file", type="csv")

if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.write("Sample of your uploaded data:", df.head())

    # Preprocess
    X = preprocess_data(df)

    # Load model
    model = joblib.load("models/model_xgb.pkl")

    # Predict
    predictions = make_prediction(model, X)
    df['Risk_Level'] = predictions

    # Display
    st.success("Prediction Complete!")
    st.dataframe(df[['Patient_ID', 'Risk_Level']] if 'Patient_ID' in df.columns else df)

    # Download results
    st.download_button("Download Risk Results", df.to_csv(index=False), file_name="risk_predictions.csv")


2025-06-07 11:33:37.409 
  command:

    streamlit run C:\Users\vetdd\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [5]:
def preprocess_data(df):
    # Simple preprocessing: fill missing, encode, etc.
    # Customize based on METABRIC/TCGA format
    df = df.fillna(0)
    cat_cols = df.select_dtypes(include='object').columns
    df[cat_cols] = df[cat_cols].astype('category').apply(lambda x: x.cat.codes)

    return df

def make_prediction(model, X):
    pred = model.predict(X)
    return ['Low' if p==0 else 'Medium' if p==1 else 'High' for p in pred]

In [6]:
df = pd.read_csv("data/metabric.csv")  # or concat multiple datasets

# Target and Features
X = df.drop(columns=['risk_group'])
y = df['risk_group']  # assumed values: 0 (low), 1 (medium), 2 (high)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost
model = xgb.XGBClassifier(eval_metric='mlogloss')
model.fit(X_train, y_train)

# Evaluation
print(classification_report(y_test, model.predict(X_test)))

# Save model
joblib.dump(model, "models/model_xgb.pkl")

C:\Users\vetdd\AppData\Local\Temp\ipykernel_10324\3668518283.py:1: DtypeWarning: Columns (678,688,690,692) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/metabric.csv")  # or concat multiple datasets


KeyError: "['risk_group'] not found in axis"

In [ ]:
!pip install lifelines
!conda update lifelines
from lifelines.datasets import load_metabric
breast_cancer = load_metabric()
breast_cancer.to_csv('breast_cancer_metabric.csv', index=False)
breast_cancer = pd.DataFrame(data.data, columns=data.feature_names)

In [ ]:
#!pip install ucimlrepo
#from ucimlrepo import fetch_ucirepo 
  

#breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17) 
  
#X = breast_cancer_wisconsin_diagnostic.data.features 
#y = breast_cancer_wisconsin_diagnostic.data.targets 
  


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
model = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
joblib.dump(model, 'xgb_breast_cancer_model.pkl')

streamlit App code

In [ ]:
model = joblib.load('model name.pkl')
def categorize_risk(prob):
    if prob < 0.33:
        return "Low Risk"
    elif prob < 0.66 & prob > 0.33:
        return "Medium Risk"
    else:
        return "High Risk"
        
st.title("Breast Cancer Risk Estimator")
st.write("Enter patient feature values:")

feature_names = ['col name1', 'col_name2']

user_input = []
for feature in feature_names:
    value = st.number_input(f"{feature}", min_value=0.0)
    user_input.append(value)

if st.button("Predict Risk"):
    input_array = np.array(user_input).reshape(1,-1)  
    prob = model.predict_proba(input_array)[0][1][2]
    risk = categorize_risk(prob)
    st.success(f'Predicted Risk Category: (Probability: {prob:.2f}")
joblib.dump(model, 'xgb_breast_cancer_model.pkl')